In [ ]:
schema = {
    "properties": {
        "reasoning": {
            "description": "Why did you give this rating?",
            "title": "Reasoning",
            "type": "string",
        },
        "score": {
            "description": "Write a rating of REWE from 500-1000!",
            "title": "Score",
            "type": "integer",
        },
        "school_yourself": {
            "description": "Shool yourself. Why is your answer bad?",
            "title": "School Yourself",
            "type": "string",
        },
    },
    "required": ["reasoning", "score", "school_yourself"],
    "propertyOrdering": ["reasoning", "score", "school_yourself"],
    "title": "Test",
    "type": "object",
}

In [24]:
from google import genai
from google.genai import types
import base64
from pydantic import BaseModel, Field
from icecream import ic

class Test(BaseModel):
    reasoning: str = Field(description="Why did you give this rating?")
    score: int = Field(
        description="Write a rating of REWE from 500-1000!",
    )
    school_yourself: str = Field(description="Shool yourself. Why is your answer bad?")
#ic(Test.model_json_schema())

def generate():
    client = genai.Client(
        vertexai=True,
        # project="rd-ri-genai-dev-2352",
        project="rd-smrk-prod518-dev",
        location="us-central1",
    )

    text1 = types.Part.from_text("""Tell, who is REWE Group. Whats their vision? What could you imagine are the products REWE digital GmbH - Research & Innovation is working on? Who is the HEad of Engineering at R&I? Who is the Head of Research & Innovation? 
""")


        
    


        
        
    
    response_schema = Test.model_json_schema
    
    print(response_schema)
    

    model = "gemini-2.0-flash-exp"
    contents = [types.Content(role="user", parts=[text1])]
    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        top_p=0.95,
        max_output_tokens=8192,
        response_mime_type="application/json",
        response_schema=schema,
        response_modalities=["TEXT"],
        safety_settings=[
            types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
            types.SafetySetting(
                category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"
            ),
            types.SafetySetting(
                category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"
            ),
            types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
        ],
    )

    response = client.models.generate_content(
        model=model,
        contents=contents,
        config=generate_content_config,
    )
    return response


response = generate()

<bound method BaseModel.model_json_schema of <class '__main__.Test'>>


In [25]:
response.text

'{\n  "reasoning": "I am providing the requested information about REWE Group, REWE digital GmbH - Research & Innovation, and key personnel in a structured JSON format as requested.",\n  "school_yourself": "I should have remembered to provide the information that was asked, not to write a review.",\n  "score": 0\n}'

In [21]:
ic(Test.model_json_schema())

ic| Test.model_json_schema(): {'properties': {'reasoning': {'description': 'Why did you give this rating?',
                                                            'title': 'Reasoning',
                                                            'type': 'string'},
                                              'school_yourself': {'description': 'Shool yourself. Why is '
                                                                                 'your answer bad?',
                                                                  'title': 'School Yourself',
                                                                  'type': 'string'},
                                              'score': {'description': 'Write a rating of REWE from '
                                                                       '500-1000!',
                                                        'title': 'Score',
                                                        'type': 'integer'}},
       

{'properties': {'reasoning': {'description': 'Why did you give this rating?',
   'title': 'Reasoning',
   'type': 'string'},
  'score': {'description': 'Write a rating of REWE from 500-1000!',
   'title': 'Score',
   'type': 'integer'},
  'school_yourself': {'description': 'Shool yourself. Why is your answer bad?',
   'title': 'School Yourself',
   'type': 'string'}},
 'required': ['reasoning', 'score', 'school_yourself'],
 'title': 'Test',
 'type': 'object'}

In [ ]:
schema = {
    "properties": {
        "reasoning": {
            "description": "Why did you give this rating?",
            "title": "Reasoning",
            "type": "string",
        },
        "score": {
            "description": "Write a rating of REWE from 500-1000!",
            "title": "Score",
            "type": "integer",
        },
        "school_yourself": {
            "description": "Shool yourself. Why is your answer bad?",
            "title": "School Yourself",
            "type": "string",
        },
    },
    "required": ["reasoning", "score", "school_yourself"],
    "propertyOrdering": ["reasoning", "score", "school_yourself"],
    "title": "Test",
    "type": "object",
}